# Market Values.

Para enriquecer nuestra base de datos, extraeremos el valor de mercado de cada jugardor de La Liga en la temporada 2019-2020.
Obtendremos esta información de www.transfermarkt.com, portal de referencia en el cálculo de esta métrica.

Sin embargo, nos encontramos con un problema:
    - No podemos acceder a una única url que contenga el valor de mercado para todos los jugadores de La Liga.
    
Lo que nos encontramos es una página en la que se muestra el valor total para cada equipo (junto con otros datos), y a partir de ahí podemos ir accediendo uno a uno a los equipos para ver el valor de cada uno de sus jugadores (https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2019).

Sí que hay una url que nos muestra el valor para todos los jugadores de La Liga, pero esta información hace referencia a la temporada actual y no a la temporada objeto de estudio en nuestro proyecto.

Esto implica que no podamos hacer un único web scrapping a partir del cual obtengamos todos los datos que nos interesan. Tendremos que hacer un scrap para cada equipo de la liga.

Selenium podría resolvernos de manera más eficiente el problema (nos permite navegar por la página web que queremos scrapear), pero en este proyecto no utilizaremos esta herramienta.

Por otro lado, no tenemos accesos a la API de Transfermarkt en la que se presupone que estarían disponibles todos estos datos. Sin embargo, existe una API generada por terceros que sí nos facilita mucho esta tarea: https://apify.com/petr_cermak/transfermarkt.

Utilizaremos esta API para acceder a la información de las diferentes páginas de www.transfermarkt.com y, como veremos más adelante, el manejo de la API es muy sencillo. Realmente se gestiona a través de una interfaz gráfica en la que incluimos la url que queremos scrapear y nos devuelve otra url (o archivo) con la información solicitada. A partir de ahí, haremos las requests y manejaremos sus respectivos json para obtener el resultado deseado.


In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

## API para el valor de mercado de los jugadores (transfermarkt).

Comenzamos por el primer equipo de La Liga según orden alfabético: Athletic Club (según la nomenclatura utilizada en Teams Stats).

Tendremos que seguir el mismo proceso para cada equipo de La Liga y sus respectivas URLs proporcionadas por la API. Podemos generar un data frame para cada uno de ellos y concatenarlos todos en un único DataFrame

In [2]:
url = 'https://api.apify.com/v2/datasets/wAt2WvPDhZ51fRFq3/items?clean=true&format=json'

In [3]:
bilbao =req.get(url)

In [4]:
bilbao.json() # Mostramos el json esta primera vez para entender mejor el proceso, pero se omitirá en el resto de los equipos.

[{'id': '621',
  'url': 'https://www.transfermarkt.com/athletic-bilbao/startseite/verein/621/saison_id/2019',
  'type': 'club',
  'squadSize': '25',
  'averageAge': '26.1',
  'foreigners': ['0', ' %'],
  'nationalTeamPlayers': '2',
  'stadium': ['San Mamés', '53.289 Seats'],
  'currentTransferRecord': '+-0',
  'players': [{'number': '1',
    'player': ['Unai Simón', 'Goalkeeper'],
    'marketValue': '€16.00m',
    'url': 'https://www.transfermarkt.com/unai-simon/profil/spieler/262396',
    'id': '262396',
    'nat.': 'Spain'},
   {'number': '13',
    'player': ['Iago Herrerín', 'Goalkeeper'],
    'marketValue': '€2.80m',
    'url': 'https://www.transfermarkt.com/iago-herrerin/profil/spieler/71570',
    'id': '71570',
    'nat.': 'Spain'},
   {'number': '31',
    'player': ['Jokin Ezkieta', 'Goalkeeper'],
    'marketValue': '€300Th.',
    'url': 'https://www.transfermarkt.com/jokin-ezkieta/profil/spieler/379548',
    'id': '379548',
    'nat.': 'Spain'},
   {'number': '35',
    'player'

In [5]:
# Observamos que el contenido que nos interesa está vinculado a la key 'players', así que estraemos este valor.

bilbao_players = []

for e in bilbao.json():
    bilbao_players.append(e['players'])

In [6]:
bilbao_players 

# Es una lista dentro de otra y esto da problemas a la hora de generar el DataFrame (nos los incluiría todo en una única fila).

[[{'number': '1',
   'player': ['Unai Simón', 'Goalkeeper'],
   'marketValue': '€16.00m',
   'url': 'https://www.transfermarkt.com/unai-simon/profil/spieler/262396',
   'id': '262396',
   'nat.': 'Spain'},
  {'number': '13',
   'player': ['Iago Herrerín', 'Goalkeeper'],
   'marketValue': '€2.80m',
   'url': 'https://www.transfermarkt.com/iago-herrerin/profil/spieler/71570',
   'id': '71570',
   'nat.': 'Spain'},
  {'number': '31',
   'player': ['Jokin Ezkieta', 'Goalkeeper'],
   'marketValue': '€300Th.',
   'url': 'https://www.transfermarkt.com/jokin-ezkieta/profil/spieler/379548',
   'id': '379548',
   'nat.': 'Spain'},
  {'number': '35',
   'player': ['Ander Iru', 'Goalkeeper'],
   'marketValue': '€50Th.',
   'url': 'https://www.transfermarkt.com/ander-iru/profil/spieler/480958',
   'id': '480958',
   'nat.': 'Spain'},
  {'number': '4',
   'player': ['Iñigo Martínez', 'Centre-Back'],
   'marketValue': '€25.50m',
   'url': 'https://www.transfermarkt.com/inigo-martinez/profil/spieler/1

In [7]:
# Aplanamos la lista y obtenemos el resultado deseado.

bilbao_def = []

for lista in bilbao_players:
    for e in lista:
        bilbao_def.append(e)

In [8]:
# Convertimos el json en DataFrame.

bilbao_df = pd.DataFrame(bilbao_def)

In [9]:
bilbao_df.head()

,number,player,marketValue,url,id,nat.
0,1,"[Unai Simón, Goalkeeper]",€16.00m,https://www.transfermarkt.com/unai-simon/profi...,262396,Spain
1,13,"[Iago Herrerín, Goalkeeper]",€2.80m,https://www.transfermarkt.com/iago-herrerin/pr...,71570,Spain
2,31,"[Jokin Ezkieta, Goalkeeper]",€300Th.,https://www.transfermarkt.com/jokin-ezkieta/pr...,379548,Spain
3,35,"[Ander Iru, Goalkeeper]",€50Th.,https://www.transfermarkt.com/ander-iru/profil...,480958,Spain
4,4,"[Iñigo Martínez, Centre-Back]",€25.50m,https://www.transfermarkt.com/inigo-martinez/p...,158863,Spain


Este es el proceso que debemos seguir para cada equipo de La Liga (al tener los datos de cada equipo en urls diferentes, tendríamos que repetir el proceso para cada url).

Esto quiere decir que, si lo hiciésemos una a una, tendríamos que repetir el proceso otras 19 veces.

Pero... ¿No podemos hacer un loop para "automatizar" este proceso?

Claro que podemos y, de hecho, es así como lo haremos.

In [10]:
# En primer lugar, necesitamos obtener todas las urls de la API y generar una lista sobre la que iterar a la hora de ejecutar las requests.
# Este proceso puede resultar tedioso, pero es necesario para mejorar la eficiencia de nuestro código a través del loop.

all_urls = ['https://api.apify.com/v2/datasets/9Msmk560b31FWcIWm/items?clean=true&format=json', # Athletic CLub
            'https://api.apify.com/v2/datasets/ZebEeRsCvEc9gD1xX/items?clean=true&format=json', # Atlético de Madrid
            'https://api.apify.com/v2/datasets/4r1kmRJfhPJKsJQOf/items?clean=true&format=json', # Leganés
            'https://api.apify.com/v2/datasets/n7sN3szXGVxKhLGZh/items?clean=true&format=json', # Alavés
            'https://api.apify.com/v2/datasets/7GWuhYdCNwGxaJb9C/items?clean=true&format=json', # FC Barcelona
            'https://api.apify.com/v2/datasets/sW4QSnkzzhWV2pRmv/items?clean=true&format=json', # Getafe
            'https://api.apify.com/v2/datasets/l5QJFc9fTv00YY2A1/items?clean=true&format=json', # Girona
            'https://api.apify.com/v2/datasets/WnkaXIh1MvhDc0VGB/items?clean=true&format=json', # Levante
            'https://api.apify.com/v2/datasets/8GWfjRoY5jcJFt6GX/items?clean=true&format=json', # Valladolid
            'https://api.apify.com/v2/datasets/alwGcp3VKntp9cX85/items?clean=true&format=json', # Rayo Vallecano
            'https://api.apify.com/v2/datasets/5O2A1uC05TEeKLUd2/items?clean=true&format=json', # Celta
            'https://api.apify.com/v2/datasets/AIBJTSpInNNitxsea/items?clean=true&format=json', # Espanyol
            'https://api.apify.com/v2/datasets/bNIQ94gzPUZHUR6BF/items?clean=true&format=json', # Betis
            'https://api.apify.com/v2/datasets/rROiXPsG09Vbd0h4q/items?clean=true&format=json', # Real Madrid
            'https://api.apify.com/v2/datasets/0tjTaZ9KswAQJrO7i/items?clean=true&format=json', # Real Sociedad
            'https://api.apify.com/v2/datasets/e8fNDHNRhCjdcD78o/items?clean=true&format=json', # Eibar
            'https://api.apify.com/v2/datasets/don47mheRvCle8Zfk/items?clean=true&format=json', # Huesca
            'https://api.apify.com/v2/datasets/8J61KF3m1e0uWymGO/items?clean=true&format=json', # Sevilla
            'https://api.apify.com/v2/datasets/Vw32sGnwnLe1njwo0/items?clean=true&format=json', # Valencia
            'https://api.apify.com/v2/datasets/QGPpZX9CYw7AgR7jA/items?clean=true&format=json', # Villareal
           ]

In [11]:
all_players = []    # Aplicamos el mismo bucle que utilizamos antes para un único equipo, pero en este caso sobre la lista de urls.
                    
for e in all_urls:
    team = req.get(e)
    team_players = []
    for i in team.json():
        team_players.append(i['players'])
        team_def = []
        for x in team_players:
            for y in x:
                team_def.append(y)
            all_players.append(team_def)

# La lista all_players es contiene listas de diccionarios (una lista por cada equipo que contiene un diccionario por jugador).

In [12]:
all_players_def = []  # Aplanamos lista para tener una lista de diccionarios y generar un DataFrame sin problemas.

for lista in all_players:
    for e in lista:
        all_players_def.append(e)

In [13]:
all_players_df = pd.DataFrame(all_players_def)  # Generamos el DataFrame.

In [14]:
all_players_df.head()

# Quizá el mayor inconveniente de este procedimiento es que sólo tenemos el nombre del jugador como identificador de cada uno.
# Este será el único valor que podemos utilizar para incorporar los valores de mercado a la tabla general de Players Stats.
# El número de dorsal también debería coincidir con el recogido en la tabla general de jugadores, pero probablemente no lo utilicemos.

,number,player,marketValue,url,id,nat.
0,25,"[Unai Simón, Goalkeeper]",€6.00m,https://www.transfermarkt.com/unai-simon/profi...,262396,Spain
1,13,"[Iago Herrerín, Goalkeeper]",€4.00m,https://www.transfermarkt.com/iago-herrerin/pr...,71570,Spain
2,1,"[Álex Remiro, Goalkeeper]",€1.50m,https://www.transfermarkt.com/alex-remiro/prof...,212862,Spain
3,26,"[Hodei Oleaga, Goalkeeper]",€200Th.,https://www.transfermarkt.com/hodei-oleaga/pro...,338286,Spain
4,4,"[Iñigo Martínez, Centre-Back]",€32.00m,https://www.transfermarkt.com/inigo-martinez/p...,158863,Spain


### Limpieza all_players_df

Lo fundamental será limpiar los valores de 'player'.

Esta columna contiene una lista con el nombre del jugador y su posición. Lo que haremos será extraer el nombre (primer elemento de la lista) y generar una nueva columna ('position') con la posición del jugador dentro del campo (segundo elemento de la lista).

In [16]:
all_players_df["position"] = all_players_df.player.apply(lambda x : x[1]) # Generamos nueva columna ('position') con la posición del jugador.

In [18]:
all_players_df["player"] = all_players_df.player.apply(lambda x : x[0]) # Hacemos que en la columna 'player' quede solo el primer elemento de la lista.

In [19]:
all_players_df.head()

,number,player,marketValue,url,id,nat.,position
0,25,Unai Simón,€6.00m,https://www.transfermarkt.com/unai-simon/profi...,262396,Spain,Goalkeeper
1,13,Iago Herrerín,€4.00m,https://www.transfermarkt.com/iago-herrerin/pr...,71570,Spain,Goalkeeper
2,1,Álex Remiro,€1.50m,https://www.transfermarkt.com/alex-remiro/prof...,212862,Spain,Goalkeeper
3,26,Hodei Oleaga,€200Th.,https://www.transfermarkt.com/hodei-oleaga/pro...,338286,Spain,Goalkeeper
4,4,Iñigo Martínez,€32.00m,https://www.transfermarkt.com/inigo-martinez/p...,158863,Spain,Centre-Back


In [20]:
# Ordenamos las columnas según el orden deseado y no incluimos id (no nos aporta valor en este caso; si necesitamos un id único distinto del nombre podremos generarlo en MySQL directamente):

all_players_df = all_players_df[['player', 'position', 'number', 'nat.', 'url', 'marketValue']]

In [21]:
all_players_df.head()

,player,position,number,nat.,url,marketValue
0,Unai Simón,Goalkeeper,25,Spain,https://www.transfermarkt.com/unai-simon/profi...,€6.00m
1,Iago Herrerín,Goalkeeper,13,Spain,https://www.transfermarkt.com/iago-herrerin/pr...,€4.00m
2,Álex Remiro,Goalkeeper,1,Spain,https://www.transfermarkt.com/alex-remiro/prof...,€1.50m
3,Hodei Oleaga,Goalkeeper,26,Spain,https://www.transfermarkt.com/hodei-oleaga/pro...,€200Th.
4,Iñigo Martínez,Centre-Back,4,Spain,https://www.transfermarkt.com/inigo-martinez/p...,€32.00m


## API para el valor total de mercado de los equipos (transfermarkt).

   Valor total por Equipos

In [ ]:
url = 'https://api.apify.com/v2/datasets/gZ9UsjGqqe8NRajlv/items?clean=true&format=json'

In [ ]:
teams_values = req.get(url)

In [ ]:
teams_values_def = []  # Accedemos a la información de clubs (key 'clubs' dentro del json).

for e in teams_values.json():
    teams_values_def.append(e['clubs'])

In [ ]:
teams_values_def2 = []

for lista in teams_values_def: # Aplanamos lista para genererar DataFrame.
    for e in lista:
        teams_values_def2.append(e)

In [ ]:
teams_values_df = pd.DataFrame(teams_values_def2)

teams_values_df.head()